# Current Blockchain ( 2nodes )

In [ ]:
!rm -f /tmp/*.json
!rm -f /tmp/*.pb

In [109]:
!peer lifecycle chaincode querycommitted --channelID mychannel --name scray-invoice-example --output json

{
	"sequence": 1,
	"version": "1.0",
	"endorsement_plugin": "escc",
	"validation_plugin": "vscc",
	"validation_parameter": "EiAvQ2hhbm5lbC9BcHBsaWNhdGlvbi9FbmRvcnNlbWVudA==",
	"collections": {},
	"approvals": {
		"Org1MSP": true,
		"Org2MSP": true,
		"klaus3MSP": false
	}
}


# init peer access (certificates,...)

In [107]:
import os
import json
import subprocess
import pandas as pd
 
pwd0 = "/home/jovyan/work/fabric-samples/"
pwd = "/home/jovyan/work/fabric-samples/test-network"

os.environ['PATH'] = "/home/jovyan/work/fabric-samples/bin:" + str(os.environ.get('PATH')) 
os.environ['CORE_PEER_TLS_ENABLED'] = "true"
os.environ['CORE_PEER_LOCALMSPID'] = "Org1MSP"
os.environ['CORE_PEER_TLS_ROOTCERT_FILE'] = pwd + "/organizations/peerOrganizations/org1.example.com/peers/peer0.org1.example.com/tls/ca.crt"
os.environ['CORE_PEER_MSPCONFIGPATH'] = pwd + "/organizations/peerOrganizations/org1.example.com/users/Admin@org1.example.com/msp"
os.environ['CORE_PEER_ADDRESS'] = "peer0.org1.example.com:7051"
os.environ['FABRIC_CFG_PATH'] = pwd0 + "/config/"

addr0 = 'peer0.org1.example.com:7050'
orderer = 'orderer.example.com'
ordererpem = pwd + '/organizations/ordererOrganizations/example.com/orderers/orderer.example.com/msp/tlscacerts/tlsca.example.com-cert.pem'
addr1 = 'peer0.org1.example.com:7051'
crt1 = pwd + '/organizations/peerOrganizations/org1.example.com/peers/peer0.org1.example.com/tls/ca.crt'
addr2 = 'peer0.org2.example.com:9051'
crt2 = pwd + '/organizations/peerOrganizations/org2.example.com/peers/peer0.org2.example.com/tls/ca.crt' 
msp1 = pwd + '/organizations/peerOrganizations/org1.example.com/users/Admin@org1.example.com/msp'
msp2 = pwd + '/organizations/peerOrganizations/org2.example.com/users/Admin@org2.example.com/msp'

def createEnvironment(orgname): 
    os.environ['ORDERER_IP']="10.15.128.97"
    os.environ['ORDERER_HOSTNAME']="orderer.example.com" 
    os.environ['CHANNEL_NAME']="mychannel"
    os.environ['ORG_ID']=orgname
    os.environ['CORE_PEER_LOCALMSPID']="Org1MSP"
    os.environ['CORE_PEER_TLS_ROOTCERT_FILE']=crt1
    os.environ['CORE_PEER_MSPCONFIGPATH']=msp1
    os.environ['ORDERER_CA']=ordererpem
    os.environ['CORE_PEER_ADDRESS'] = "peer0.org1.example.com:7051"

In [108]:
createEnvironment('klaus3')
!/home/jovyan/work/scray/projects/invoice-hyperledger-fabric/scripts/inform_existing_nodes_local.sh $ORDERER_IP $CHANNEL_NAME $ORG_ID "/home/jovyan/work/fabric-samples/test-network" 

2020-12-17 10:09:04.696 UTC [channelCmd] InitCmdFactory -> INFO 001 Endorser and orderer connections initialized
2020-12-17 10:09:04.701 UTC [cli.common] readBlock -> INFO 002 Received block: 5
2020-12-17 10:09:04.701 UTC [channelCmd] fetch -> INFO 003 Retrieving last config block: 2
2020-12-17 10:09:04.704 UTC [cli.common] readBlock -> INFO 004 Received block: 2
<!DOCTYPE HTML PUBLIC "-//IETF//DTD HTML 2.0//EN">
<html><head>
<title>301 Moved Permanently</title>
</head><body>
<h1>Moved Permanently</h1>
<p>The document has moved <a href="http://10.15.136.41:30080/ca/">here</a>.</p>
</body></html>
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 11419  100 11419    0     0  2787k      0 --:--:-- --:--:-- --:--:-- 2787k
2020-12-17 10:09:05.086 UTC [channelCmd] InitCmdFactory -> INFO 001 Endorser and orderer connections initialized
2020-12-17 10:09:05.150 UTC [channelCmd] InitCm

/home/jovyan/work/scray/projects/invoice-hyperledger-fabric/python
2020-12-15 15:03:15.080 UTC [channelCmd] InitCmdFactory -> INFO 001 Endorser and orderer connections initialized
2020-12-15 15:03:15.088 UTC [cli.common] readBlock -> INFO 002 Received block: 6
2020-12-15 15:03:15.089 UTC [channelCmd] fetch -> INFO 003 Retrieving last config block: 6
2020-12-15 15:03:15.092 UTC [cli.common] readBlock -> INFO 004 Received block: 6
<!DOCTYPE HTML PUBLIC "-//IETF//DTD HTML 2.0//EN">
<html><head>
<title>301 Moved Permanently</title>
</head><body>
<h1>Moved Permanently</h1>
<p>The document has moved <a href="http://10.15.136.41:30080/ca/">here</a>.</p>
</body></html>
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   236  100   236    0     0  59000      0 --:--:-- --:--:-- --:--:-- 59000
parse error: Invalid numeric literal at line 1043, column 10
configtxlator: error: Error deco

In [ ]:
#!more ./new_member_org.json



In [29]:
!/home/jovyan/work/scray/projects/invoice-hyperledger-fabric/scripts/example_network_install_and_approve_cc.sh 10.14.128.38 10.15.128.97 /home/jovyan/work/fabric-samples/test-network

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   450  100   450    0     0   1636      0 --:--:-- --:--:-- --:--:--  1636
2020-12-16 13:26:01.357 UTC [cli.lifecycle.chaincode] submitInstallProposal -> INFO 001 Installed remotely: response:<status:200 payload:"\nJbasic_1.0:5a294a12a1a89cd4eed3d4234fbc79f42eab2ac20cd176bc8ebbc07c597cd0ee\022\tbasic_1.0" > 
2020-12-16 13:26:01.357 UTC [cli.lifecycle.chaincode] submitInstallProposal -> INFO 002 Chaincode code package identifier: basic_1.0:5a294a12a1a89cd4eed3d4234fbc79f42eab2ac20cd176bc8ebbc07c597cd0ee
2020-12-16 13:26:01.460 UTC [cli.lifecycle.chaincode] submitInstallProposal -> INFO 001 Installed remotely: response:<status:200 payload:"\nJbasic_1.0:5a294a12a1a89cd4eed3d4234fbc79f42eab2ac20cd176bc8ebbc07c597cd0ee\022\tbasic_1.0" > 
2020-12-16 13:26:01.460 UTC [cli.lifecycle.chaincode] submitInstallProposal -> INFO 002 Chainco

In [91]:
def getCrt(org='',domain='kubernetes.research.dev.seeburger.de',host='10.15.136.41:30080',user='scray', pwd='scray',channel='mychannel'):
    try:    
        output = subprocess.check_output(['curl', '--user',user + ':' + pwd, 
                                               'http://' + host + '/ca/' + channel + '/' + org + '-' + domain + '-ca.crt'])
        #return str(callProcess)[3:-3]
        return output.decode('ascii')
    except Exception as e:
        return str(e)

def getCrt(org='',domain='kubernetes.research.dev.seeburger.de',host='10.15.136.41:30080',user='scray', pwd='scray',channel='mychannel'):
    try:    
        output = subprocess.check_output(['curl', '--user',user + ':' + pwd, 
                                               'http://' + host + '/ca/' + channel + '/' + org + '-' + domain + '-ca.crt', '> /tmp/new.crt'])
        #return str(callProcess)[3:-3]
        return output.decode('ascii')
    except Exception as e:
        return str(e)    
    
    
#klaus-kubernetes.research.dev.seeburger.de-ca.crt    

In [ ]:
peer lifecycle chaincode approveformyorg 
-o orderer.example.com:7050 
--ordererTLSHostnameOverride orderer.example.com 
--tls  
--cafile /tmp/tlsca.example.com-cert.pem 
--channelID $CHANNEL_ID 
--name basic 
--version 1.0 --package-id $PKGID --sequence 1

In [93]:
#print(getCrt(org='klaus'))

def commit():
    try:    
        #crtnew = '\" + getCrt(org='klaus') + '\"
        getCrt(org='klaus')
        output = subprocess.check_output(['peer', 'lifecycle','chaincode','commit','-o',addr0,'--ordererTLSHostnameOverride',orderer,
                                          '--tls','--cafile',ordererpem,'--channelID','mychannel','--name','basic',
                                          '--peerAddresses',addr1, '--tlsRootCertFiles', crt1, 
                                          '--peerAddresses', addr2, '--tlsRootCertFiles', crt2, 
                                          '--peerAddresses', 'kubernetes.research.dev.seeburger.de:32590', '--tlsRootCertFiles', '/tmp/new.crt',
                                          '--version', '1.0', '--sequence', '1','--tlsRootCertFiles'])
        return output.decode('ascii')
    except Exception as e:
        return str(e)
    
commit()    

"Command '['peer', 'lifecycle', 'chaincode', 'commit', '-o', 'peer0.org1.example.com:7050', '--ordererTLSHostnameOverride', 'orderer.example.com', '--tls', '--cafile', '/home/jovyan/work/fabric-samples/test-network/organizations/ordererOrganizations/example.com/orderers/orderer.example.com/msp/tlscacerts/tlsca.example.com-cert.pem', '--channelID', 'mychannel', '--name', 'basic', '--peerAddresses', 'peer0.org1.example.com:7051', '--tlsRootCertFiles', '/home/jovyan/work/fabric-samples/test-network/organizations/peerOrganizations/org1.example.com/peers/peer0.org1.example.com/tls/ca.crt', '--peerAddresses', 'peer0.org2.example.com:9051', '--tlsRootCertFiles', '/home/jovyan/work/fabric-samples/test-network/organizations/peerOrganizations/org2.example.com/peers/peer0.org2.example.com/tls/ca.crt', '--peerAddresses', 'kubernetes.research.dev.seeburger.de:32590', '--tlsRootCertFiles', '/tmp/new.crt', '--version', '1.0', '--sequence', '1', '--tlsRootCertFiles']' returned non-zero exit status 1."

In [95]:
getCrt(org='klaus')
!ls /tmp

config.json	   npm-1340-763f38f4  v8-compile-cache-1000
core-js-banners    npm-1382-9afeabcc  yarn--1604853375593-0.12874406731735522
hsperfdata_jovyan  npm-1424-58869de0  yarn--1604853379789-0.7799701827508732
hsperfdata_root    npm-67-0cb1e20a    yarn--1604853380954-0.47958525452069933


In [22]:
os.environ['IP_CC_SERVICE']='10.14.128.38'
#Host where the example network is running
os.environ['IP_OF_EXAMPLE_NETWORK']='10.15.128.97'

!/home/jovyan/work/scray/projects/invoice-hyperledger-fabric/scripts/example_network_commit_cc.sh $IP_CC_SERVICE $IP_OF_EXAMPLE_NETWORK /home/jovyan/work/fabric-samples/test-network

2020-12-16 12:54:21.506 UTC [chaincodeCmd] ClientWait -> INFO 001 txid [f84b8a5386703833d028daa338b9e4c5433d336e232cd3f8d304e034a4807358] committed with status (ENDORSEMENT_POLICY_FAILURE) at peer0.org1.example.com:7051
2020-12-16 12:54:21.511 UTC [chaincodeCmd] ClientWait -> INFO 002 txid [f84b8a5386703833d028daa338b9e4c5433d336e232cd3f8d304e034a4807358] committed with status (ENDORSEMENT_POLICY_FAILURE) at peer0.org2.example.com:9051
Error: transaction invalidated with status (ENDORSEMENT_POLICY_FAILURE)
Error: endorsement failure during invoke. response: status:500 message:"make sure the chaincode basic has been successfully defined on channel mychannel and try again: chaincode basic not found" 


In [26]:
!cat /home/jovyan/work/scray/projects/invoice-hyperledger-fabric/scripts/example_network_commit_cc.sh

EXT_CC_IP=$1
HOST_IP=$2
CERT_BASE_PATH=$3

export PKGID=basic_1.0:5a294a12a1a89cd4eed3d4234fbc79f42eab2ac20cd176bc8ebbc07c597cd0ee
#CERT_BASE_PATH=/opt/gopath/src/github.com/hyperledger/fabric/peer

export CORE_PEER_TLS_ENABLED=true
export ORDERER_CA=${CERT_BASE_PATH}/organizations/ordererOrganizations/example.com/orderers/orderer.example.com/msp/tlscacerts/tlsca.example.com-cert.pem
export PEER0_ORG1_CA=${CERT_BASE_PATH}/organizations/peerOrganizations/org1.example.com/peers/peer0.org1.example.com/tls/ca.crt
export PEER0_ORG2_CA=${CERT_BASE_PATH}/organizations/peerOrganizations/org2.example.com/peers/peer0.org2.example.com/tls/ca.crt

#CERT_BASE_PATH=/opt/gopath/src/github.com/hyperledger/fabric/peer

export CORE_PEER_TLS_ENABLED=true
export ORDERER_CA=${CERT_BASE_PATH}/organizations/ordererOrganizations/example.com/orderers/orderer.example.com/msp/tlscacerts/tlsca.example.com-cert.pem
export PEER0_ORG1_CA=${CERT_BASE_PATH}/organizations/peerOrganizations/org1.example.com/peers/peer0.

# old

In [ ]:

!peer channel fetch config /tmp/config_block.pb -o orderer.example.com:7050 -c $CHANNEL_NAME --tls --cafile $ORDERER_CA > /dev/null
!configtxlator proto_decode --input /tmp/config_block.pb --type common.Block | jq .data.data[0].payload.data.config > /tmp/config.json   

In [ ]:
# Upload CA cert
os.environ['SHARED_FS_HOST']="10.15.136.41:30080"
os.environ['SHARED_FS_USER']="scray"
os.environ['SHARED_FS_PW']="scray"
#!apk add curl

os.environ['NEW_ORG_NAME']="OrgSally"

!curl --user $SHARED_FS_USER:$SHARED_FS_PW -X MKCOL http://$SHARED_FS_HOST/ca
!curl --user $SHARED_FS_USER:$SHARED_FS_PW -T /home/jovyan/work/fabric-samples/test-network/organizations/ordererOrganizations/example.com/orderers/orderer.example.com/msp/tlscacerts/tlsca.example.com-cert.pem http://$SHARED_FS_HOST/ca/tlsca.example.com-cert.pem

# Get configuration of new peer
#curl --user 'scray:scray' http://${SHARED_FS_HOST}/newmemberrequests/$CHANNEL_NAME/${NEW_ORG_NAME}.json > new_member_org.json
!curl --user $SHARED_FS_USER:$SHARED_FS_PW  http://${SHARED_FS_HOST}/newmemberrequests/$CHANNEL_NAME/${NEW_ORG_NAME}.json > /tmp/new_member_org.json 
        
# Add org3 data to existing config
!jq -s ".[0] "*" {\"channel_group\":{\"groups\":{\"Application\":{\"groups\": {\"${NEW_ORG_NAME}MSP\":.[1]}}}}}" /tmp/config.json /tmp/new_member_org.json > /tmp/modified_config.json

!configtxlator proto_encode --input /tmp/config.json --type common.Config --output /tmp/config.pb
!configtxlator proto_encode --input /tmp/modified_config.json --type common.Config --output /tmp/modified_config.pb
!configtxlator compute_update --channel_id $CHANNEL_NAME --original /tmp/config.pb --updated /tmp/modified_config.pb --output /tmp/org3_update.pb
!configtxlator proto_decode --input /tmp/org3_update.pb --type common.ConfigUpdate | jq . > /tmp/org3_update.json
!echo '{"payload":{"header":{"channel_header":{"channel_id":"'$CHANNEL_NAME'", "type":2}},"data":{"config_update":'$(cat /tmp/org3_update.json)'}}}' | jq . > /tmp/org3_update_in_envelope.json
!configtxlator proto_encode --input /tmp/org3_update_in_envelope.json --type common.Envelope --output /tmp/org3_update_in_envelope.pb        

sign and update

In [ ]:
!peer channel signconfigtx -f /tmp/org3_update_in_envelope.pb

os.environ['CORE_PEER_LOCALMSPID']="Org2MSP"
os.environ['CORE_PEER_TLS_ROOTCERT_FILE']=crt2
os.environ['CORE_PEER_MSPCONFIGPATH']=msp2
os.environ['ORDERER_CA']=ordererpem
os.environ['CORE_PEER_ADDRESS'] = addr2

!peer channel update -f /tmp/org3_update_in_envelope.pb -c $CHANNEL_NAME -o orderer.example.com:7050 --tls --cafile $ORDERER_CA

# network_install_and_approve_cc (3 nodes)

In [110]:
#!pwd
#!ls /home/jovyan/work/scray/projects/invoice-hyperledger-fabric/scripts/example_network_install_and_approve_cc.sh
!cd /tmp
!/home/jovyan/work/scray/projects/invoice-hyperledger-fabric/scripts/example_network_install_and_approve_cc.sh 10.14.128.38 10.15.128.97 /home/jovyan/work/fabric-samples/test-network

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   450  100   450    0     0   3488      0 --:--:-- --:--:-- --:--:--  3488
2020-12-17 10:49:00.800 UTC [cli.lifecycle.chaincode] submitInstallProposal -> INFO 001 Installed remotely: response:<status:200 payload:"\nJbasic_1.0:5a294a12a1a89cd4eed3d4234fbc79f42eab2ac20cd176bc8ebbc07c597cd0ee\022\tbasic_1.0" > 
2020-12-17 10:49:00.801 UTC [cli.lifecycle.chaincode] submitInstallProposal -> INFO 002 Chaincode code package identifier: basic_1.0:5a294a12a1a89cd4eed3d4234fbc79f42eab2ac20cd176bc8ebbc07c597cd0ee
2020-12-17 10:49:00.901 UTC [cli.lifecycle.chaincode] submitInstallProposal -> INFO 001 Installed remotely: response:<status:200 payload:"\nJbasic_1.0:5a294a12a1a89cd4eed3d4234fbc79f42eab2ac20cd176bc8ebbc07c597cd0ee\022\tbasic_1.0" > 
2020-12-17 10:49:00.902 UTC [cli.lifecycle.chaincode] submitInstallProposal -> INFO 002 Chainco

# commit

In [111]:
os.environ['IP_CC_SERVICE']='10.14.128.38'
#Host where the example network is running
os.environ['IP_OF_EXAMPLE_NETWORK']='10.15.128.97'

!/home/jovyan/work/scray/projects/invoice-hyperledger-fabric/scripts/example_network_commit_cc.sh $IP_CC_SERVICE $IP_OF_EXAMPLE_NETWORK /home/jovyan/work/fabric-samples/test-network

2020-12-17 10:49:32.095 UTC [chaincodeCmd] ClientWait -> INFO 001 txid [5daa75d059206312dfbe203f680817ee66e55c4b11ee9222cb79424c4eb345da] committed with status (VALID) at peer0.org2.example.com:9051
2020-12-17 10:49:32.101 UTC [chaincodeCmd] ClientWait -> INFO 002 txid [5daa75d059206312dfbe203f680817ee66e55c4b11ee9222cb79424c4eb345da] committed with status (VALID) at peer0.org1.example.com:7051
Error: endorsement failure during invoke. response: status:500 message:"error in simulation: failed to execute transaction 218a5afe598ea0af612d90bfe617254a75e9154283c5f51174f19eb93e7c4ffc: could not launch chaincode basic_1.0:5a294a12a1a89cd4eed3d4234fbc79f42eab2ac20cd176bc8ebbc07c597cd0ee: connection to basic_1.0:5a294a12a1a89cd4eed3d4234fbc79f42eab2ac20cd176bc8ebbc07c597cd0ee failed: error cannot create connection for basic_1.0:5a294a12a1a89cd4eed3d4234fbc79f42eab2ac20cd176bc8ebbc07c597cd0ee: error creating grpc connection to asset-transfer-basic.org1.example.com:30999: failed to create new co

# check network (approved chaincode)

In [101]:
!peer lifecycle chaincode querycommitted --channelID mychannel --name basic --output json

{
	"sequence": 1,
	"version": "1.0",
	"endorsement_plugin": "escc",
	"validation_plugin": "vscc",
	"validation_parameter": "EiAvQ2hhbm5lbC9BcHBsaWNhdGlvbi9FbmRvcnNlbWVudA==",
	"collections": {},
	"approvals": {
		"Org1MSP": true,
		"Org2MSP": true,
		"klausMSP": true
	}
}


In [106]:
!peer lifecycle chaincode querycommitted --channelID mychannel --name scray-invoice-example --output json

{
	"sequence": 1,
	"version": "1.0",
	"endorsement_plugin": "escc",
	"validation_plugin": "vscc",
	"validation_parameter": "EiAvQ2hhbm5lbC9BcHBsaWNhdGlvbi9FbmRvcnNlbWVudA==",
	"collections": {},
	"approvals": {
		"Org1MSP": true,
		"Org2MSP": true
	}
}


# Test

In [ ]:
def readAsset(asset='asset1'):
    callopt= '{\"function\":\"ReadAsset\",\"Args\":[' + '\"' + asset + '\"' + ']}'
    try:    
        callProcess  = subprocess.check_output(['peer', 'chaincode','query', '-C', 'mychannel', '-n', 'basic', '-c', callopt])
        return json.loads(str(callProcess)[2:-3])
    except Exception as e:
        return str(e)
    
readAsset('asset1')    

In [ ]:
readAsset('asset2')  

In [ ]:
readAsset('asset4')  

In [ ]:
!peer chaincode query -C mychannel -n basic -c '{"function":"GetAllAssets","Args":[]}'

# REST

In [ ]:
!peer channel fetch config /tmp/config_block.pb -o orderer.example.com:7050 -c $CHANNEL_NAME --tls --cafile $ORDERER_CA > /dev/null  

In [ ]:
#peer channel fetch config config_block.pb -o orderer.example.com:7050 -c $CHANNEL_NAME --tls --cafile $ORDERER_CA
import sys    
def channelFetch(channelName,ordererpem):
    #rint(channelName,ordererpem)
    try:    
        output = subprocess.check_output(['peer', 'channel','fetch', 'config', 'config_block.pb', '-o', 'orderer.example.com:7050', '-c',channelName, '--tls', 
                                               '--cafile', ordererpem], stderr=subprocess.STDOUT, shell=True, timeout=3,universal_newlines=True)
        #output = subprocess.check_output(['peer', 'channel','fetch', 'config', 'config_block.pb', '-o', 'orderer.example.com:7050', '-c',channelName, '--tls', 'cafile', ordererpem], stderr=subprocess.STDOUT, shell=True, timeout=3,universal_newlines=True)
        #output = subprocess.check_output(['peer', 'channel','fetch','newest', 'config', '/tmp/config_block.pb', '-o', 'orderer.example.com:7050', '-c',channelName, '--tls', '--cafile', ordererpem], stderr=subprocess.STDOUT)
        
    #xcept Exception as e:
    #   print('exception' + str(e))
    #   return None
    except subprocess.CalledProcessError as exc:
        print("Status : FAIL", exc.returncode, exc.output)
    else:
        print("Output: \n{}\n".format(output))
    
#channelFetch('mychannel',ordererpem)    

#!cd /home/jovyan/work/
#rm -f config_block.pb
channelFetch('mychannel',ordererpem)
